In [ ]:
import pandas as pd

df = pd.read_csv("./script/FONSEScripMaster.txt", dtype=str)

data = df[["Token", "InstrumentName", "ShortName", "Series", "ExpiryDate", "StrikePrice", "OptionType", "BasePrice", "ExchangeCode"]]

data = data[data["Series"] == "OPTION"]
data = data[data["InstrumentName"] == "OPTSTK"]

# Print first few rows
print(data.head())

# only when it's needed to get latest data from the broker
# unique_instruments = data["ExchangeCode"].unique()
# for instrument in unique_instruments:
#     instrument_data = data[data["ExchangeCode"] == instrument]
#     file_name = f"./Optiondata/{instrument}.xlsx"
#     instrument_data.to_excel(file_name, index = False)


     Token InstrumentName ShortName  Series   ExpiryDate StrikePrice  \
797  35172         OPTSTK    ADAENT  OPTION  24-Apr-2025        3000   
799  35175         OPTSTK    ADAENT  OPTION  24-Apr-2025        3000   
832  35233         OPTSTK    ADAPOR  OPTION  24-Apr-2025        1600   
833  35234         OPTSTK    ADAPOR  OPTION  24-Apr-2025        1600   
835  35238         OPTSTK    AARIND  OPTION  27-Mar-2025         515   

    OptionType BasePrice ExchangeCode  
797         CE    229010     ADANIENT  
799         PE    229010     ADANIENT  
832         CE    109960   ADANIPORTS  
833         PE    109960   ADANIPORTS  
835         CE     37600     AARTIIND  


In [ ]:
import threading
from flask import Flask, request, jsonify, redirect, session
from breeze_connect import BreezeConnect
import urllib.parse

app = Flask(__name__)
app.secret_key = "1997"

API_KEY = "4_8463B9784119l72yC4426912px346P"
API_SECRET = "816O1S=eG215716g518CM)X225Z0!78b"
ACCESS_TOKEN = None

breeze = BreezeConnect(api_key=API_KEY)

@app.route('/')
def login():
    login_url = f"https://api.icicidirect.com/apiuser/login?api_key={urllib.parse.quote_plus(API_KEY)}"
    return redirect(login_url)

@app.route('/callback', methods=['POST'])
def callback():
    global ACCESS_TOKEN

    session_token = request.args.get("apisession")
    if not session_token:
        return "Authorization failed", 400

    try:
        breeze.generate_session(api_secret=API_SECRET, session_token=session_token)
        ACCESS_TOKEN = session_token
        session["access_token"] = ACCESS_TOKEN
        
        # Fetch customer details
        customer_details = breeze.get_customer_details(api_session=ACCESS_TOKEN)
        return jsonify({"message": "Session active", "customer_details": customer_details})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Start Flask in a separate thread
def run_flask():
    app.run(debug=True, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2025 11:45:56] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [13/Mar/2025 11:46:51] "POST /callback?apisession=50877563 HTTP/1.1" 200 -


###### 
* product_type:- "futures", "options", "furtureplus", "futureplus_sltp", "optionplus", "cash", "eatm", "btst", "margin", "marginplus"
* exchange_code:- "NSE", "NFO", "BSE"
* right:- 	"call", "put", "others"


In [39]:
from datetime import datetime
import pandas as pd

# Function to format date and datetime in ISO format
def format_iso(date_str, time_str="07:00:00"):
    return f"{datetime.strptime(date_str, '%d/%m/%Y').date()}T{time_str}.000Z"

# Define required dates
start_date = format_iso("28/08/2024", "09:15:00")
end_date = format_iso("29/08/2024", "15:30:00")
exp_date = format_iso("29/08/2024", "15:30:00")

# Other parameters
stock_code = "ICIBAN"
exchange_code = "NFO"
interval = "1minute"
strike_price = "1200"
product_type = "options"
rights = ["call", "put"]

all_data = []  # List to store data for both "call" and "put"

# Fetch data for both call and put options
for right in rights:
    data = breeze.get_historical_data(
        interval=interval,
        from_date=start_date,
        to_date=end_date,
        expiry_date=exp_date,
        stock_code=stock_code,
        strike_price=strike_price,
        product_type=product_type,
        exchange_code=exchange_code,
        right=right
    )
    
    if "Success" in data and isinstance(data["Success"], list):  # Ensure valid response
        df = pd.DataFrame(data["Success"])  # Convert API response to DataFrame
        all_data.append(df)

# Combine both DataFrames into one
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)  # Merge call & put data
    
    # Save to Excel
    final_df.to_excel("ICICI.xlsx", index=False, engine="openpyxl")
    print("Data successfully saved to ICICI.xlsx")
else:
    print("No valid data received.")


Data successfully saved to ICICI.xlsx
